## Importing necessary libraries

In [1]:
from Google import Create_Service
import re
from pprint import pprint
from googleapiclient import errors

import pandas as pd
import numpy as np

In [2]:
from googleapiclient.discovery import build

import os
import dotenv

dotenv.load_dotenv()

key = os.environ.get('API_KEY')

In [3]:
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

In [ ]:
scopes = ["https://www.googleapis.com/auth/youtube.readonly"]

In [ ]:
api_service_name = "youtube"
api_version = "v3"
client_secrets_file = "client_secret.json"

In [ ]:
# Get credentials and create an API client
flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
    client_secrets_file, scopes)

In [ ]:
credentials = flow.run_console()

In [ ]:
youtube = googleapiclient.discovery.build(
        api_service_name, api_version, credentials=credentials)

In [4]:
youtube = build('youtube',
                'v3',
                developerKey=key)

In [ ]:
request = youtube.search().list(
        part='snippet',
        q='how to',
        maxResults=50
)

In [ ]:
response = request.execute()

In [ ]:
response.keys()

In [ ]:
response['items'][0]['snippet'].keys()

In [ ]:
response['items'][0]['snippet']

In [ ]:
len(response['items'])

In [ ]:
channel_id = ''

items = []

In [ ]:
for item in response['items']:
    item = item['snippet']['channelId']
    if item not in items:
        items.append(item)

In [ ]:
len(items)

In [ ]:
items

In [ ]:
','.join(items)

In [ ]:
response['nextPageToken']

In [ ]:
response['items'][0].keys()

In [ ]:
response

In [ ]:
pprint(response['items'][0]['snippet']['channelId'])

In [7]:
subs = youtube.channels().list(
    part='contentDetails',
    id='UC295-Dw_tDNtZXFeAPAW6Aw,UCJtS_7gQr-mpwDvmPODptPg,UCUsN5ZwHx2kILm84-jPDeXw,UCKA00zb2VKPHIR4iRpYWQfQ,UCU_W0oE_ock8bWKjALiGs8Q,UCIOE_IzglfnHyaOYbvpJxjA,UCke6I9N4KfC968-yRcd5YRg,UCEr55381WIqO1w_IzgcI5DQ,UCzrMFWFWPou9RCMufXwWdDg'
    ).execute()

In [13]:
subs['items'][0].keys()

dict_keys(['kind', 'etag', 'id', 'contentDetails'])

In [12]:
subs['items'][0]

{'kind': 'youtube#channel',
 'etag': 'ZHrEgMcrJ7TsbKLWufiV1QrBHmk',
 'id': 'UC295-Dw_tDNtZXFeAPAW6Aw',
 'contentDetails': {'relatedPlaylists': {'likes': '',
   'uploads': 'UU295-Dw_tDNtZXFeAPAW6Aw'}}}

In [8]:
subs

{'kind': 'youtube#channelListResponse',
 'etag': 'w_r-e3AVAGqiogzjw-nDwChyVUI',
 'pageInfo': {'totalResults': 9, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#channel',
   'etag': 'ZHrEgMcrJ7TsbKLWufiV1QrBHmk',
   'id': 'UC295-Dw_tDNtZXFeAPAW6Aw',
   'contentDetails': {'relatedPlaylists': {'likes': '',
     'uploads': 'UU295-Dw_tDNtZXFeAPAW6Aw'}}},
  {'kind': 'youtube#channel',
   'etag': 'CPmEnMS8XEt9d6QeygbzWoCY7Bg',
   'id': 'UCJtS_7gQr-mpwDvmPODptPg',
   'contentDetails': {'relatedPlaylists': {'likes': '',
     'uploads': 'UUJtS_7gQr-mpwDvmPODptPg'}}},
  {'kind': 'youtube#channel',
   'etag': 'UTo6j7haxPl_T2NsUh2282kFMEc',
   'id': 'UCU_W0oE_ock8bWKjALiGs8Q',
   'contentDetails': {'relatedPlaylists': {'likes': '',
     'uploads': 'UUU_W0oE_ock8bWKjALiGs8Q'}}},
  {'kind': 'youtube#channel',
   'etag': 'wePNM8MSjRceZl03ke32RKw1iVc',
   'id': 'UCIOE_IzglfnHyaOYbvpJxjA',
   'contentDetails': {'relatedPlaylists': {'likes': '',
     'uploads': 'UUIOE_IzglfnHyaOYbvpJxjA'}}},
  {'kind

In [ ]:
subs['items'][0]['statistics']['subscriberCount']

In [ ]:
if 1000 < 1500 < 10000:
    print('hi')

In [ ]:
status = youtube.channels().list(
    part='contentOwnerDetails',
    id='UCVpjwUjHY7VRZ_ILIPuarRg'
    ).execute()

In [ ]:
status

In [ ]:
{'kind': 'youtube#channelListResponse',
 'etag': 'DQQ8Mw8qxFYgusNP4jXSOF8h69k',
 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#channel',
   'etag': '0vaUrbl4ySogbZ-RYYDUVggK0eo',
   'id': 'UCzav3AnB8UKKM6zzg1_CHpA',
   'contentOwnerDetails': {}}]}

In [ ]:
subs_2 = youtube.subscriptions().list(
    part='statistics',
    id='UCzav3AnB8UKKM6zzg1_CHpA'
    ).execute()

In [ ]:
# Creating YouTube class to communicate with YouTube API
class YouTube:
    """
    Communicate with YouTube API

    ...

    Attributes:
        key: str
            Api key used to create service and authenticate user

    Methods:
        construct_service():
            Construct service using API_KEY
        upload_response():
            Retrieve all uploaded videos playlist's ID
        get_playlist_items():
            Retrieve all videos information from playlist
    """
    def __init__(self, key, scopes: list = None):
        # self.secret_file = secret_file
        self.key = key
        self.scopes = scopes

    # def construct_service(self):
    #     """
    #         Responsible for creating service instance from 'google.Create_Service'
    #     """
    #     API_SERVICE = 'youtube'
    #     API_VERSION = 'v3'
    #     service = Create_Service(self.secret_file, API_SERVICE, API_VERSION, self.scopes)
    #     return service

    def construct_service(self):
        """
        Creates service object from build method
        """

        API_SERVICE = 'youtube'
        API_VERSION = 'v3'
        service = build(
            API_SERVICE,
            API_VERSION,
            developerKey=self.key
        )
        return service

    @staticmethod
    def upload_response(service, channel_id: str) -> str:
        """
        Send request to retrieve uploaded videos response as playlist ID.

            Parameters:
                service: Instance of Create_Service()
                    service object created using construct_service()
                channel_id: str
                    Channel's id required for request
            Returns:
                str: playlist_id
        """
        request = service.channels().list(
            part='contentDetails',
            id=channel_id
        )

        response = request.execute()  # Send request and receive response

        # Extract playlist_id from the received response
        playlist_id = response['items'][0]['contentDetails']['relatedPlaylists']['uploads']

        return playlist_id

    @staticmethod
    def get_playlist_items(service, playlist_Id: str):
        """
        Retrieve all videos information from playlist.

        Parameters:
            service: Instance of Create_Service()
                service object created using construct_service()
            playlist_Id: str
                Id of the playlist from which to retrieve data

        Returns:
            List: contains information of all videos
        """
        try:
            # Create request to retrieve playlist items
            request = service.playlistItems().list(
                part='contentDetails',
                playlistId=playlist_Id,
                maxResults=50  # Max results per request (maximum: 50)
            )

            response = request.execute()  # Send request and receive response

            #print(response)

            playlist_items = response['items']  # Grabs only videos info from the response
            nextPageToken = response['nextPageToken']  # Grabs next page token

            print(nextPageToken)

            current_page = 1

            # Retrieve data while the next page is available
            while nextPageToken:
                request = service.playlistItems().list(
                    part='contentDetails',
                    playlistId=playlist_Id,
                    maxResults=50,  # max results per request (maximum: 50)
                    pageToken=nextPageToken
                )

                response = request.execute()  # Send request

                print(f'Current Page: {current_page}')  # prints current page
                current_page += 1

                # Add items to playlist_items that are retrieved from next page
                playlist_items.extend(response['items'])
                nextPageToken = response.get('nextPageToken')

                #print(response)
                print(nextPageToken)
                
            print(f'Total Videos found: {len(playlist_items)}')
                
        except KeyError as e:
            print(f'Total Videos found: {len(playlist_items)}')
            pass

        videos_id = []  # Holds all available videos id's

        # Go through playlist items list and retrieve all videos id's
        for video_id in playlist_items:
            try:
                id = video_id['snippet']['resourceId']['videoId']
                videos_id.append(id)

            except KeyError:
                id = video_id['contentDetails']['videoId']
                videos_id.append(id)

        videos_info = []  # Holds info about all available videos

        # Loop through all videos id's and retrieve info
        for batch_num in range(0, len(videos_id), 50):
            # Create batches of videos to request data
            videos_batch = videos_id[batch_num: batch_num + 50]  # Batch Size: 50

            # Send request to retrieve video's details
            response_videos = service.videos().list(
                # video details to be retrieved for each video
                part='contentDetails,snippet,statistics',
                id=videos_batch,
                maxResults=50
            ).execute()
            # batch items received from videos response
            batch_items = response_videos['items']
            # Adding batch items to videos_info list
            videos_info.extend(batch_items)

        return videos_info

    @staticmethod
    def convert_duration_to_seconds(duration: str) -> int:
        """
        Converts video duration to seconds

        Parameters:
            duration: str ->.
                time duration in format '00H00M00S'

        Returns:
            int: total number of seconds
        """

        h = int(re.search('\d+H', duration)[0][:-1]) * 60**2 if re.search('\d+H', duration) else 0
        m = int(re.search('\d+M', duration)[0][:-1]) * 60 if re.search('\d+M', duration) else 0
        s = int(re.search('\d+S', duration)[0][:-1]) if re.search('\d+S', duration) else 0
        return h + m + s

    @staticmethod
    def create_csv(data):
        for item in data:
            title = item['snippet']['title']
            print(title)

In [ ]:
yt = YouTube(key)

In [ ]:
service = yt.construct_service()

In [ ]:
service

In [ ]:
response = yt.upload_response(service, 'UCSJBJ3sP5GRUJMON12v28ew')

In [ ]:
response

In [ ]:
items = yt.get_playlist_items(service, 'UUSJBJ3sP5GRUJMON12v28ew')

In [ ]:
type(items)

In [ ]:
from pprint import pprint

pprint(items[0])

In [ ]:
items[0]['statistics']['dislikeCount']

In [ ]:
title = items[0]['snippet']['title']
title

In [ ]:
date = items[0]['snippet']['publishedAt']
date[:10]

In [ ]:
duration = items[0]['contentDetails']['duration']
duration

In [ ]:
views = items[0]['statistics']['viewCount']
views

In [ ]:
yt.convert_duration_to_seconds(duration)

In [ ]:
import re

In [ ]:
int(re.search('\d+S', 'PT42M39S')[0][:-1]) if re.search('\d+S', 'PT42M39S') else 0

In [ ]:
int(re.search('\d+M', 'PT42M39S')[0][:-1]) * 60 if re.search('\d+M', 'PT42M39S') else 0

In [ ]:
int(re.search('\d+H', 'PT42M39S')[0][:-1]) * 60**2 if re.search('\d+H', 'PT42M39S') else 0

In [ ]:
yt.create_csv(items)

In [ ]:
titles, dates, views, durations = [], [], [], []

for item in items:
    title = item['snippet']['title']
    date = item['snippet']['publishedAt'][:10]
    view = item['statistics']['viewCount']
    duration = item['contentDetails']['duration']
    duration = YouTube.convert_duration_to_seconds(duration)

    titles.append(title)
    dates.append(date)
    views.append(view)
    durations.append(duration)
    
    
len(titles), len(dates), len(views), len(durations)

In [ ]:
yt_data = pd.DataFrame({
    'Title': titles,
    'Upload_Date': dates,
    'Views': views,
    'Duration': durations
})

In [ ]:
yt_data.head()

In [ ]:
yt_data.to_csv('data.csv', index=False)

In [ ]:
titles, dates, views, durations, likes, dislikes, comments = ([] for _ in range(7))


In [ ]:
id(titles), id(dates)

In [ ]:
type(titles), type(dates)

In [ ]:
id(ls1), id(ls2)

In [ ]:
columns = ['titles', 'dates', 'views', 'durations', 'likes', 'dislikes', 'comments']

In [ ]:
for _ in columns:
    globals()[_] = []
    print(_)
    print(type(_))

In [ ]:
type('titles')

In [ ]:
exec([] for _ in columns)